In [10]:
import pandas as pd
import seaborn as sb

#df = pd.read_csv('forestfires.csv')
df_portugal = pd.read_csv('data/Portugal_Dataset.csv')
df_portugal
#sb.heatmap(df.corr(), annot = True, fmt='.1g')


,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,4,3,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44
513,2,4,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29
514,7,4,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16
515,1,4,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00


In [11]:
# Remove irrelevant columns
# Location may be irrelevant because all are xy coordinates for Montesinho park
columns_to_keep_portugal = [
    'month', 
    'day', 
    'temp', 
    'RH',  # relative humidity
    'wind',  # wind speed
    'rain',  # rain amount
    'area'  # burned area
]

df_portugal = df_portugal[columns_to_keep_portugal]
df_portugal.shape


(517, 7)

In [12]:
# Removing all missing data
df_portugal = df_portugal.dropna()
print(df_portugal.shape)

(517, 7)


In [13]:
# Show updated dataset
df_portugal
df_portugal.to_csv('data/Portugal_Dataset_clean.csv')

In [14]:
import pandas as pd
import seaborn as sb

df_cal_weather = pd.read_csv('data/California_Weather.csv' )

#df_cal_weather

In [15]:
# Remove irrelevant columns
columns_to_keep_cal_weather = [
    'Stn Name',
    'CIMIS Region', 
    'Date', 
    'Precip (in)',
    #'Max Air Temp (F)',
    #'Min Air Temp (F)',
    'Avg Air Temp (F)',
    #'Max Rel Hum (%)',
    #'Min Rel Hum (%)',
    'Avg Rel Hum (%)',
    'Avg Wind Speed (mph)',
    'Wind Run (miles)'
]

df_cal_weather = df_cal_weather[columns_to_keep_cal_weather]
df_cal_weather.shape

(128125, 8)

In [16]:
# Removing all missing data
df_cal_weather = df_cal_weather.dropna()
print(df_cal_weather.shape)

(128112, 8)


In [17]:
# Show updated dataset
df_cal_weather
df_cal_weather.to_csv('data/California_Weather_clean.csv')

In [18]:
import pandas as pd
import seaborn as sb

df_cal_fire = pd.read_csv('data/California_Fire_Incidents.csv')
#df_cal_fire


In [19]:
# Remove irrelevant columns
columns_to_keep_cal_fire = [
    'AcresBurned',
    'ArchiveYear',
    'CalFireIncident',
    'Counties', 
    'Extinguished',
    'Location', 
    'MajorIncident', 
    'Started'
]

df_cal_fire = df_cal_fire[columns_to_keep_cal_fire]
df_cal_fire.shape

(1636, 8)

In [20]:
# Removing all missing data
df_cal_fire = df_cal_fire.dropna()
print(df_cal_fire.shape)

(1577, 8)


In [21]:
# Show updated dataset
df_cal_fire
df_cal_fire.to_csv('data/California_Fire_Incidents_clean.csv')